In [ ]:
# 目标 熟悉 langchain 调用大模型
# 1.在 LangChain 中使用 LLM
# 2.使用 prompt templates 组件
# 3.使用 LangSmith 调试和追踪

In [2]:
# 安装依赖
%uv add langchain

Note: you may need to restart the kernel to use updated packages.


Resolved 111 packages in 2ms
Audited 106 packages in 0.34ms


In [ ]:
# 相关环境变量设置
import sys 
sys.path.append("..") 
from config import config_loader

config_loader.load_env()

In [5]:
# 安装谷歌聊天模型 可以根据自己的情况自行选择
%uv add -U langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


Resolved 125 packages in 2.38s
 Downloaded grpcio
 Downloaded google-ai-generativelanguage
Prepared 14 packages in 4.33s
         If the cache and target directories are on different filesystems, hardlinking may not be supported.
         If this is intentional, set `export UV_LINK_MODE=copy` or use `--link-mode=copy` to suppress this warning.
Installed 14 packages in 3.05s
 + cachetools==5.5.2
 + filetype==1.2.0
 + google-ai-generativelanguage==0.6.17
 + google-api-core==2.24.2
 + google-auth==2.38.0
 + googleapis-common-protos==1.69.2
 + grpcio==1.71.0
 + grpcio-status==1.71.0
 + langchain-google-genai==2.1.1
 + proto-plus==1.26.1
 + protobuf==5.29.4
 + pyasn1==0.6.1
 + pyasn1-modules==0.4.1
 + rsa==4.9


In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage

# 配置谷歌聊天模型
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

# SystemMessage: 启动 AI 行为的信息，作为消息序列的第一个传入输入消息
# HumanMessage: 用户传入的消息
messages = [
    SystemMessage("把下面的中文翻译成英文"),
    HumanMessage("你是谁？"),
]
ai_msg = model.invoke(messages)
print(ai_msg)
print(ai_msg.content)

content='Who are you?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-bb9dc6d5-cdec-4cce-8e79-258f5be240b1-0' usage_metadata={'input_tokens': 9, 'output_tokens': 5, 'total_tokens': 14, 'input_token_details': {'cache_read': 0}}
Who are you?


In [7]:
# 发送消息调用模型 下面三种方式是等价的
model.invoke("Hello")

model.invoke([{"role": "user", "content": "Hello"}])

model.invoke([HumanMessage("Hello")])

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-0820f90d-305d-41fb-bf8f-a888d386b324-0', usage_metadata={'input_tokens': 1, 'output_tokens': 10, 'total_tokens': 11, 'input_token_details': {'cache_read': 0}})

In [8]:
# 消息流式传输 下面将会一个token一个token的输出，并且不同token之间以 | 分隔
for token in model.stream(messages):
    print(token.content, end="|")

Who| are you?|

In [9]:
# 提示词模板
# 定义一套可复用的提示词模板，可以用提示词模板结构化提示词，将提示词分为系统指令和用户输入两部分，易于理解和维护，并且可传参，动态替换提示词模板中的变量
# 可复用性、结构化提示词、灵活性
from langchain_core.prompts import ChatPromptTemplate

system_template = "把这段文本翻译成{language}"
# 定义系统角色和用户角色各自的发言
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)
# 在这里传参
prompt = prompt_template.invoke({
    "language": "英文",
    "text": "你好呀",
})
print(prompt)
print(prompt.to_messages())
# 测试模型调用
ai_msg = model.invoke(prompt)
ai_msg.content

messages=[SystemMessage(content='把这段文本翻译成英文', additional_kwargs={}, response_metadata={}), HumanMessage(content='你好呀', additional_kwargs={}, response_metadata={})]
[SystemMessage(content='把这段文本翻译成英文', additional_kwargs={}, response_metadata={}), HumanMessage(content='你好呀', additional_kwargs={}, response_metadata={})]


'Here are a few options, depending on the nuance you want to convey:\n\n*   **Hello there!** (Friendly and common)\n*   **Hi!** (Simple and informal)\n*   **Hey!** (Casual)\n*   **Hi there!** (Slightly more enthusiastic than "Hi")'